In [5]:
import keras
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential,load_model
import tensorflow.keras.backend as K
from commai_spd_chl_keras_datagen import commai_speed_chl_gen
from tensorflow.keras.layers import Dense,Activation,Lambda,GRU,BatchNormalization,Activation,Flatten,Conv2D,MaxPooling2D,Reshape,Dropout

In [6]:
tf.config.list_physical_devices ()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
tf.keras.backend.image_data_format()

'channels_last'

In [8]:
seq = Sequential()

#should I trust this guy?https://www.kdnuggets.com/2018/09/dropout-convolutional-networks.html

seq.add(Conv2D(filters=32,kernel_initializer='he_normal', kernel_size=(4,4),padding='same',input_shape=(480,640,3)))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Conv2D(filters=32, kernel_size=(4,4),kernel_initializer='he_normal',padding='same'))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Conv2D(filters=64, kernel_size=(4,4),kernel_initializer='he_normal',padding='same'))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Conv2D(filters=64, kernel_size=(4,4),kernel_initializer='he_normal',padding='same'))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Flatten())

seq.add(Dense(192,kernel_initializer='he_normal'))
seq.add(Activation("relu"))
seq.add(BatchNormalization())
seq.add(Dropout(.3)) # throw away 30%

seq.add(Dense(50,kernel_initializer='he_normal'))
seq.add(Activation("relu"))
seq.add(Dropout(.2))

seq.add(Dense(1,kernel_initializer='he_normal'))

seq.compile(loss='mse', optimizer='adam')
seq.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 480, 640, 32)      1568      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 120, 160, 32)      0         
_________________________________________________________________
activation_6 (Activation)    (None, 120, 160, 32)      0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 120, 160, 32)      128       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 120, 160, 32)      16416     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 30, 40, 32)        0         
_________________________________________________________________
activation_7 (Activation)    (None, 30, 40, 32)       

In [9]:
batch_size=45
n_e=60
n_frames_train=15000
n_frames_test=2900
n_frames_valid=2500

train_idx,test_idx,valid_idx=commai_speed_chl_gen(n_frames_valid=n_frames_valid,frames_per_sample=2,mode="optical_flow_dense",
                    n_frames_test=n_frames_test,n_frames_train=n_frames_train,batch_size=batch_size).get_indexes()

#load=np.load(open("idxes_45bs_opflowdense_15k.np","rb"))
#train_idx=load["train_idx"]
#test_idx=load["test_idx"]
#valid_idx=load["valid_idx"]

train_gen=commai_speed_chl_gen(frame_idx=train_idx,batch_size=batch_size,mode="optical_flow_dense",frames_per_sample=2)
test_gen=commai_speed_chl_gen(frame_idx=test_idx,batch_size=batch_size,mode="optical_flow_dense",frames_per_sample=2)
valid_gen=commai_speed_chl_gen(frame_idx=valid_idx,batch_size=batch_size,mode="optical_flow_dense",frames_per_sample=2)

np.savez_compressed(open("idxes_45bs_opflowdense_15k.np","wb+"),train_idx=train_idx,test_idx=test_idx,valid_idx=valid_idx)



(480, 640, 3)
(45, 480, 640, 3)
(45, 1)
train_samples: 7500 
test_samples:1450 
valid_samples: 1250
(480, 640, 3)
(45, 480, 640, 3)
(45, 1)
(480, 640, 3)
(45, 480, 640, 3)
(45, 1)
(480, 640, 3)
(45, 480, 640, 3)
(45, 1)


In [10]:
ie=0
filepath="STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-{epoch:02d}-{val_loss:.2f}-{loss:.2f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False)

tb=tf.keras.callbacks.TensorBoard(
    log_dir='STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense', histogram_freq=1, write_graph=True, write_images=False,
    update_freq=10, profile_batch=0, embeddings_freq=0
)

cbs=[checkpoint,tb]
hst=seq.fit_generator(train_gen, epochs=n_e,workers=25,max_queue_size=85,
        use_multiprocessing=True,validation_data=test_gen,callbacks=cbs,verbose=1,shuffle=True, initial_epoch=ie)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/60
166/166 [==============================] - ETA: 0s - loss: 101.3400WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00001: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-01-51.11-101.34.hdf5
166/166 [==============================] - 259s 2s/step - loss: 101.3400 - val_loss: 51.1053
Epoch 2/60
166/166 [==============================] - ETA: 0s - loss: 48.1421WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00002: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-02-25.17-48.14.hdf5
166/166 [==============================] - 272s 2s/step - loss: 48.1421 - val_loss: 25.1656
Epoch 3/60
166/166 [===========

166/166 [==============================] - 278s 2s/step - loss: 18.3787 - val_loss: 12.9103
Epoch 9/60
166/166 [==============================] - ETA: 0s - loss: 16.6338WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00009: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-09-9.64-16.63.hdf5
166/166 [==============================] - 268s 2s/step - loss: 16.6338 - val_loss: 9.6418
Epoch 10/60
166/166 [==============================] - ETA: 0s - loss: 14.7766WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00010: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-10-12.62-14.78.hdf5
166/166 [==============================] - 263s 2s/step - loss: 14.7766 - val_loss: 12.6177
Epoch 11/60
166/166

166/166 [==============================] - ETA: 0s - loss: 9.2322WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00017: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-17-10.71-9.23.hdf5
166/166 [==============================] - 275s 2s/step - loss: 9.2322 - val_loss: 10.7136
Epoch 18/60
166/166 [==============================] - ETA: 0s - loss: 9.4079WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00018: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-18-9.54-9.41.hdf5
166/166 [==============================] - 271s 2s/step - loss: 9.4079 - val_loss: 9.5417
Epoch 19/60
166/166 [==============================] - ETA: 0s - loss: 8.9338WARNING:tensorflow:multiprocessing can interact bad

166/166 [==============================] - ETA: 0s - loss: 7.8011WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00025: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-25-12.58-7.80.hdf5
166/166 [==============================] - 258s 2s/step - loss: 7.8011 - val_loss: 12.5760
Epoch 26/60
166/166 [==============================] - ETA: 0s - loss: 7.3189WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00026: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-26-10.07-7.32.hdf5
166/166 [==============================] - 265s 2s/step - loss: 7.3189 - val_loss: 10.0660
Epoch 27/60
166/166 [==============================] - ETA: 0s - loss: 6.6305WARNING:tensorflow:multiprocessing can interact b

Process Keras_worker_ForkPoolWorker-3005:
Process Keras_worker_ForkPoolWorker-3006:
Process Keras_worker_ForkPoolWorker-3013:
Process Keras_worker_ForkPoolWorker-3001:
Process Keras_worker_ForkPoolWorker-3014:
Process Keras_worker_ForkPoolWorker-3003:
Process Keras_worker_ForkPoolWorker-3017:
Process Keras_worker_ForkPoolWorker-3016:
Process Keras_worker_ForkPoolWorker-3002:
Process Keras_worker_ForkPoolWorker-3008:
Process Keras_worker_ForkPoolWorker-3004:
Process Keras_worker_ForkPoolWorker-3007:
Process Keras_worker_ForkPoolWorker-3011:
Process Keras_worker_ForkPoolWorker-3019:
Process Keras_worker_ForkPoolWorker-3024:
Process Keras_worker_ForkPoolWorker-3022:
Process Keras_worker_ForkPoolWorker-3010:
Process Keras_worker_ForkPoolWorker-3025:
Process Keras_worker_ForkPoolWorker-3015:
Process Keras_worker_ForkPoolWorker-3020:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most rece

  File "/home/q/spd_chl/commai_spd_chl_keras_datagen.py", line 149, in opticalFlowDense
    0)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/queues

KeyboardInterrupt
  File "/home/q/spd_chl/lib/python3.6/site-packages/tensorflow/python/keras/utils/data_utils.py", line 662, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "/home/q/spd_chl/commai_spd_chl_keras_datagen.py", line 232, in __getitem__
    batch_x[sample]=self.opticalFlowDense(frames[0],frames[1])
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))


KeyboardInterrupt: 

In [11]:
#30 Epoches
seq=load_model("STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-30-9.68-6.53.hdf5")
seq.evaluate_generator(valid_gen,n_frames_valid/batch_size/2, max_queue_size = 80,workers = 10,verbose=2)

Instructions for updating:
Please use Model.evaluate, which supports generators.
28/27 - 25s - loss: 10.2564


10.256434440612793

In [13]:
#29 Epoches
seq=load_model("STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-29-7.70-6.64.hdf5")
seq.evaluate_generator(valid_gen,n_frames_valid/batch_size/2, max_queue_size = 80,workers = 10,verbose=2)

28/27 - 26s - loss: 8.4895


8.489483833312988

In [14]:
#19 Epoches
seq=load_model("STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-19-8.48-8.93.hdf5")
seq.evaluate_generator(valid_gen,n_frames_valid/batch_size/2, max_queue_size = 80,workers = 10,verbose=2)

28/27 - 28s - loss: 8.9630


8.963047981262207